<a href="https://colab.research.google.com/github/adeandak/ai-tensorflow/blob/main/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single de ABBA
BY: @adeandak, @sl-abreu, @luisfm @elmario
---



## Imports

In [ ]:
# library for understanding music
from music21 import *
# library for listing down the file names
import os
# librrary for Array Processing
import numpy as np
# importing library
from collections import Counter
# library for visualization
import matplotlib.pyplot as plt
# library for defining train and test data sets
from sklearn.model_selection import train_test_split
# keras Libraries for model
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.utils import np_utils
import keras.backend as K
#loading best model
from keras.models import load_model
import random
#tensorflow for the text processing
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
#library for measuring time
import time

# Text Generation


## Reading and Processing the data

In [ ]:
path_to_file = '/content/abba.txt' #hay que agregar el archivo cada que nos conectamos al Runtime
# leer y decodificar el archivo txt 
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#para checar que si leyo el txt vamos a imprimir los primeros 300 chars
print(text[:300])
# buscar los chars unicos, para generar palabras. 
vocab = sorted(set(text))

Mamma Mia

I've been cheated by you since I don't know when
So I made up my mind, it must come to an end
Look at me now, will I ever learn?
I don't know how but I suddenly lose control
There's a fire within my soul
Just one look and I can hear a bell ring
One more look and I forget everything, w-o-o


In [ ]:
#cell by addaku
#darle ejemplo de tokens
example_texts = ['abcdefg', 'xyz']
#procesamos el texto
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
#buscamos ahora si con nuestro input
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))
#buscamos los ids
ids = ids_from_chars(chars)
#empezamos a hacer capas con Keras
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)
#buscamos los chars desde sus ids
chars = chars_from_ids(ids)

#para armar strings
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
seq_length = 100
ids_dataset = tf.data.Dataset.from_tensor_slices(ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')))
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

### Create training batches

In [ ]:
dataset = sequences.map(split_input_target)

# GIVEN NUMS
BATCH_SIZE = 32
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

## Building the model

In [ ]:
#por convencion esto se queda asi
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch, sequence, vocab)")

(32, 100, 66) # (batch, sequence, vocab)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  16896     
_________________________________________________________________
gru_1 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_1 (Dense)              multiple                  67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [ ]:
#podemos usar esto gracias a Keras
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

#we will optimize to get the min loss
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (32, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.19034


Optimize and configure checkpoints 


In [ ]:
model.compile(optimizer='adam', loss=loss, metrics=["accuracy"]) #try different optimizers

checkpoint_callback=ModelCheckpoint('/content/text_ckpt.h5', monitor='loss', mode='min', save_best_only=True,verbose=1)
history = model.fit(dataset, epochs=80, callbacks=[checkpoint_callback])

Epoch 1/80
19/19 [==============================] - 6s 75ms/step - loss: 4.1756 - accuracy: 0.1231

Epoch 00001: loss improved from inf to 3.95746, saving model to /Users/robot/Downloads\text_ckpt.h5
Epoch 2/80
19/19 [==============================] - 2s 82ms/step - loss: 3.1982 - accuracy: 0.1640

Epoch 00002: loss improved from 3.95746 to 3.09876, saving model to /Users/robot/Downloads\text_ckpt.h5
Epoch 3/80
19/19 [==============================] - 2s 77ms/step - loss: 2.8465 - accuracy: 0.2515

Epoch 00003: loss improved from 3.09876 to 2.76182, saving model to /Users/robot/Downloads\text_ckpt.h5
Epoch 4/80
19/19 [==============================] - 2s 80ms/step - loss: 2.5342 - accuracy: 0.3131

Epoch 00004: loss improved from 2.76182 to 2.47616, saving model to /Users/robot/Downloads\text_ckpt.h5
Epoch 5/80
19/19 [==============================] - 2s 76ms/step - loss: 2.3447 - accuracy: 0.3344

Epoch 00005: loss improved from 2.47616 to 2.32533, saving model to /Users/robot/Downloa

## Use model to generate text, and a function that is explained in the ppts


In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=.5):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

def makeLyrics(aux, num):
  start = time.time()
  states = None
  next_char = tf.constant(aux)
  result = [next_char]

  for n in range(num):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  end = time.time()
  for i in range(len(next_char)):
    print(result[i].numpy().decode('utf-8'), '\n')

  print('_'*80+'\nRun time:', end - start)

# Music Generation

## Auxiliary functions

In [ ]:
#defining function to read MIDI files
def read_midi(file):
    
    print("Loading Music File:",file)
    
    notes=[]
    notes_to_parse = None
    
    #parsing a midi file
    midi = converter.parse(file)
  
    #grouping based on different instruments
    insts = instrument.partitionByInstrument(midi)

    if insts:
      notes_to_parse=insts.parts[0].recurse()
    else:
      notes_to_parse=midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

In [ ]:
def convert_to_midi(prediction_output,filepath):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.AcousticGuitar()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.4
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filepath)

## Import training data

Las primeras dos celdas de este apartado corresponden a la carga de los archivos de audio, proceso que puede tomar varios minutos. Para ahorrar tiempo se comentaron estas secciones y se reemplaza por una carga de las notas ya procesadas en un archivo .npy

In [ ]:
#specify the path
#path='/Users/robot/Downloads/music-20210430T003347Z-001/music/'

#read all the filenames
#files=[i for i in os.listdir(path) if (i.endswith(".mid") | i.endswith(".midi"))]

#reading each midi file
#notes_array = np.array([read_midi(path+i) for i in files])

Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Chiquitita.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Dame dame dame.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Dancing Queen.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Fernando.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Gimme! Gimme! Gimme!.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Mamma Mia.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Money Money Money.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/SOS.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/SuperTrouper.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z-001/music/Take A Chance On Me.mid
Loading Music File: /Users/robot/Downloads/music-20210430T003347Z

C:\Users\robot\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
#print(notes_array)
#print(notes_array.shape)

# save numpy array as npy file
#from numpy import asarray
#from numpy import save
# save to npy file
#save('/Users/robot/Downloads/notes.npy', notes_array)

[array(['A3', 'C#4', 'B3', ..., '1', 'A3', 'D5'], dtype='<U15')
 array(['6.7.10.11.0', 'A5', 'G4', ..., 'E-2', 'D3', 'E2'], dtype='<U23')
 array(['D7', 'B6', '4.5.9', ..., 'B5', 'B5', 'B5'], dtype='<U17')
 array(['A5', 'F#5', 'F#5', ..., '4.8', '4.8', 'E4'], dtype='<U17')
 array(['F#6', 'D6', 'F#6', ..., 'E-2', 'E-2', 'E-2'], dtype='<U14')
 array(['D6', 'D5', 'D3', ..., 'D3', 'A6', 'A6'], dtype='<U12')
 array(['4', 'E5', '4', ..., 'B2', 'B-2', 'B-2'], dtype='<U17')
 array(['5.9.0', '3.9', 'A6', ..., '9.1', 'D5', 'A6'], dtype='<U15')
 array(['G4', '0.4', 'C5', ..., 'C5', 'C2', 'C5'], dtype='<U15')
 array(['A3', 'G4', 'E5', ..., 'F2', 'E3', 'E3'], dtype='<U13')
 array(['F#5', 'C#7', 'G6', ..., 'G5', 'G5', 'A5'], dtype='<U19')
 array(['E-6', 'C7', 'C#4', ..., 'B4', 'B6', 'B5'], dtype='<U17')
 array(['7.9.1', 'A4', '7.9.10.0.2', ..., 'E5', 'E5', 'B5'], dtype='<U17')
 array(['11.1.6', 'G4', '2.3', ..., 'A5', 'F#1', 'B5'], dtype='<U19')]
(14,)


## Read previously imported data from npy file

In [ ]:
# load numpy array from npy file
from numpy import load

original_load=load
load = lambda *a,**k: original_load(*a, allow_pickle=True, **k)

# load array
notes_array = load('/content/notes.npy')
# print the array
print(notes_array)

load=original_load

[array(['A3', 'C#4', 'B3', ..., '1', 'A3', 'D5'], dtype='<U15')
 array(['6.7.10.11.0', 'A5', 'G4', ..., 'E-2', 'D3', 'E2'], dtype='<U23')
 array(['D7', 'B6', '4.5.9', ..., 'B5', 'B5', 'B5'], dtype='<U17')
 array(['A5', 'F#5', 'F#5', ..., '4.8', '4.8', 'E4'], dtype='<U17')
 array(['F#6', 'D6', 'F#6', ..., 'E-2', 'E-2', 'E-2'], dtype='<U14')
 array(['D6', 'D5', 'D3', ..., 'D3', 'A6', 'A6'], dtype='<U12')
 array(['4', 'E5', '4', ..., 'B2', 'B-2', 'B-2'], dtype='<U17')
 array(['5.9.0', '3.9', 'A6', ..., '9.1', 'D5', 'A6'], dtype='<U15')
 array(['G4', '0.4', 'C5', ..., 'C5', 'C2', 'C5'], dtype='<U15')
 array(['A3', 'G4', 'E5', ..., 'F2', 'E3', 'E3'], dtype='<U13')
 array(['F#5', 'C#7', 'G6', ..., 'G5', 'G5', 'A5'], dtype='<U19')
 array(['E-6', 'C7', 'C#4', ..., 'B4', 'B6', 'B5'], dtype='<U17')
 array(['7.9.1', 'A4', '7.9.10.0.2', ..., 'E5', 'E5', 'B5'], dtype='<U17')
 array(['11.1.6', 'G4', '2.3', ..., 'A5', 'F#1', 'B5'], dtype='<U19')]


## Data filtering and preparation

In [ ]:
#converting 2D array into 1D array
notes_ = [element for note_ in notes_array for element in note_]

#No. of unique notes
unique_notes = list(set(notes_))
print(len(unique_notes))

2779


In [ ]:
freq = dict(Counter(notes_))
# Frequency parameter
frr = 75
frequent_notes = [note_ for note_, count in freq.items() if count>=frr]
print(len(frequent_notes))

# Save the frequent notes in a new array
new_music=[]

for notes in notes_array:
    temp=[]
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)            
    new_music.append(temp)
    
new_music = np.array(new_music)

204


C:\Users\robot\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
tsteps = 32
x = []
y = []

for note_ in new_music:
    for i in range(0, len(note_) - tsteps, 1):
        
        #preparing input and output sequences
        input_ = note_[i:i + tsteps]
        output = note_[i + tsteps]
        
        x.append(input_)
        y.append(output)
        
x = np.array(x)
y = np.array(y)

In [ ]:
print(x.shape)
print(y.shape)

(95861, 32)
(95861,)


In [ ]:
# Assign an integer to each note
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

#preparing input sequences
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.reshape(x_seq,(len(x),tsteps,1))

unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np_utils.to_categorical([y_note_to_int[i] for i in y])

In [ ]:
#igual que en pcp
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)
print(x_tr.shape)
print(y_tr.shape)

(76688, 32, 1)
(76688, 204)


## Super Keras time

In [ ]:
musicModel = Sequential()
musicModel.add(LSTM(
  512,
  input_shape=(x_tr.shape[1],x_tr.shape[2]),
  return_sequences=True
))
musicModel.add(LSTM(256))
musicModel.add(Dense(256))
musicModel.add(Dense(len(unique_x)))
musicModel.add(Activation('softmax'))

#ada=tf.keras.optimizers.Adadelta(
#    learning_rate=0.005, rho=0.95, epsilon=1e-10,name='Adadelta')
musicModel.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

musicModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 512)           1052672   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 204)               52428     
_________________________________________________________________
activation (Activation)      (None, 204)               0         
Total params: 1,958,348
Trainable params: 1,958,348
Non-trainable params: 0
_________________________________________________________________


In [ ]:
mc=ModelCheckpoint('/content/best_model.h5', monitor='loss', mode='min', save_best_only=True,verbose=1) 

## Model Training

In [ ]:
batch = 32
epoch = 200
history = musicModel.fit(x_tr,y_tr,batch_size=batch,epochs=epoch, callbacks=[mc])

Epoch 1/200
2397/2397 [==============================] - 53s 22ms/step - loss: 1.9431 - accuracy: 0.5485

Epoch 00001: loss improved from 1.97904 to 1.94313, saving model to /Users/robot/Downloads\best_model.h5
Epoch 2/200
2397/2397 [==============================] - 53s 22ms/step - loss: 1.9023 - accuracy: 0.5540

Epoch 00002: loss improved from 1.94313 to 1.90226, saving model to /Users/robot/Downloads\best_model.h5
Epoch 3/200
2397/2397 [==============================] - 54s 22ms/step - loss: 1.8683 - accuracy: 0.5602

Epoch 00003: loss improved from 1.90226 to 1.86826, saving model to /Users/robot/Downloads\best_model.h5
Epoch 4/200
2397/2397 [==============================] - 53s 22ms/step - loss: 1.8364 - accuracy: 0.5667

Epoch 00004: loss improved from 1.86826 to 1.83642, saving model to /Users/robot/Downloads\best_model.h5
Epoch 5/200
2397/2397 [==============================] - 54s 22ms/step - loss: 1.7987 - accuracy: 0.5753

Epoch 00005: loss improved from 1.83642 to 1.79867

In [ ]:
musicModel.evaluate(x_val,y_val)

600/600 [==============================] - 6s 9ms/step - loss: 10.3402 - accuracy: 0.0175


[10.340234756469727, 0.01752464473247528]

## Prediction

In [ ]:
def makeMusic(n):
  # Loading the best model
  musicModel = Sequential()
  musicModel.add(LSTM(
    512,
    input_shape=(x_tr.shape[1],x_tr.shape[2]),
    return_sequences=True
  ))
  musicModel.add(LSTM(256))
  musicModel.add(Dense(256))
  musicModel.add(Dense(len(unique_x)))
  musicModel.add(Activation('softmax'))

  musicModel.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

  musicModel.load_weights('/content/accuracy67.h5')


  ind = np.random.randint(0,len(x_val)-1)

  random_music = x_val[ind]

  predictions=[]
  for i in range(n):

      random_music = np.reshape(random_music,(1,tsteps,1))

      prob  = musicModel.predict(random_music)
      y_pred= np.argmax(prob)
      predictions.append(y_pred)

      random_music = np.append(random_music,y_pred)
      random_music = random_music[1:tsteps+1]
      
  print(predictions)
  return predictions

In [ ]:
predictions=makeMusic(500)
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

## Convert predictions back to midi audio file

In [ ]:
convert_to_midi(predicted_notes,'/content/AIBBA5.mid')

# Our ABBA song

## Lyrics

In [ ]:
aux=['Andy ', 'Silver ', 'LuisFer ', 'Mario' ]*2
makeLyrics(aux, 250)

Andy e on the ground
Andante, Andante
Oh please don't let me down

Make me sing, make me sound
(You make me sing and you make me...)
Andante, Andante
Tread lightly on my ground
Andante, Andante
Oh please don't let me down 

You and I can share the silence 

Silver rieve in fairy-tales
Sweet nothings in my ear
But I do believe in sympathy
Gotta have love to carry on living
Gotta have love 'till eternity

People need hope, people need loving
People need trust from a fellow man
People need love to make a good liv 

LuisFer and night like this
Now I see them clearly, the things that I miss
Oh no no no no

Ah-ha-ha, ah-aaaah
Ah-ha-ha, ah-aaaah

Ah-ha-ha, keep thinking 'bout his angeleyes
I keep thinking, a-aaah

Sometimes when I'm love 's you honey)
Take a chance on me

 

Marioan
I wouldn't help it, it had to be you and I
Always thought you knew the reason why
I only wanted a little love affair
Now I can see you are beginning to care
But baby, believe me
It's better to forget me

Men are t

## Music

In [ ]:
predictions=makeMusic(500)
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]
convert_to_midi(predicted_notes,'/content/AIBBA7.mid')

[126, 95, 70, 91, 6, 18, 123, 76, 76, 18, 172, 167, 5, 120, 172, 150, 161, 17, 2, 28, 187, 167, 169, 64, 18, 155, 77, 104, 24, 142, 35, 184, 44, 17, 107, 151, 5, 59, 196, 195, 35, 137, 74, 23, 96, 6, 117, 76, 142, 31, 88, 143, 48, 181, 88, 10, 57, 24, 181, 18, 127, 181, 57, 64, 22, 123, 143, 138, 88, 126, 181, 143, 134, 85, 96, 142, 35, 200, 27, 104, 161, 54, 76, 142, 31, 159, 51, 91, 1, 187, 134, 51, 143, 161, 125, 24, 124, 68, 123, 24, 145, 81, 52, 125, 52, 126, 181, 74, 29, 187, 174, 51, 153, 117, 57, 30, 176, 125, 184, 167, 141, 70, 111, 198, 151, 104, 9, 201, 81, 145, 153, 81, 143, 126, 126, 202, 71, 202, 143, 202, 177, 177, 81, 143, 143, 81, 17, 183, 52, 186, 72, 92, 101, 9, 179, 71, 81, 145, 123, 78, 81, 52, 10, 86, 104, 159, 168, 67, 133, 7, 177, 17, 94, 94, 44, 191, 72, 81, 136, 150, 202, 189, 202, 67, 67, 185, 49, 191, 39, 18, 27, 43, 55, 181, 161, 17, 29, 101, 167, 101, 18, 7, 54, 91, 29, 29, 77, 57, 177, 177, 177, 143, 18, 63, 63, 176, 173, 143, 188, 143, 24, 183, 123, 24, 